In [ ]:
import random
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam

In [ ]:
filename = "/content/junglebook.txt"
joined_text = open(filename, 'r', encoding='utf-8').read()
joined_text = joined_text.lower()

In [ ]:
partial_text = joined_text[:100000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [ ]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [ ]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [ ]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dropout

In [ ]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(CuDNNLSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))
model.summary()

Model: "sequential"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 cu_dnnlstm (CuDNNLSTM)      (None, 10, 128)           1348096   

                                                                 

 cu_dnnlstm_1 (CuDNNLSTM)    (None, 128)               132096    

                                                                 

 dense (Dense)               (None, 2503)              322887    

                                                                 

 activation (Activation)     (None, 2503)              0         

                                                                 


Total params: 1803079 (6.88 MB)

Trainable params: 1803079 (6.88 MB)

Non-trainable params: 0 (0.00 Byte)

_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=100, shuffle=True).history

Epoch 1/100

151/151 [==============================] - 10s 12ms/step - loss: 6.4703 - accuracy: 0.0657

Epoch 2/100

151/151 [==============================] - 2s 11ms/step - loss: 6.1321 - accuracy: 0.0665

Epoch 3/100

151/151 [==============================] - 1s 9ms/step - loss: 6.0933 - accuracy: 0.0665

Epoch 4/100

151/151 [==============================] - 1s 9ms/step - loss: 6.0255 - accuracy: 0.0665

Epoch 5/100

151/151 [==============================] - 1s 9ms/step - loss: 5.9760 - accuracy: 0.0680

Epoch 6/100

151/151 [==============================] - 2s 10ms/step - loss: 5.9138 - accuracy: 0.0770

Epoch 7/100

151/151 [==============================] - 1s 10ms/step - loss: 5.8383 - accuracy: 0.0823

Epoch 8/100

151/151 [==============================] - 1s 9ms/step - loss: 5.7612 - accuracy: 0.0844

Epoch 9/100

151/151 [==============================] - 1s 10ms/step - loss: 5.6879 - accuracy: 0.0861

Epoch 10/100

151/151 [==============================] - 2s 11ms/st

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
possible = predict_next_word("Mowgli was running in the ", 5)

1/1 [==============================] - 0s 389ms/step


In [ ]:
for idx in possible:
    print(unique_tokens[idx])

my

he

mowgli

the

a


In [ ]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [ ]:
generate_text("Mowgli and bagheera were playing in the forest and then suddenly sher khan appeared ", 100, 10)

'Mowgli and bagheera were playing in the forest and then suddenly sher khan appeared fleas nor illo leap away hold foolishness uncomfortable fours crack sullenly hurried result rain yards when coping have very akela scratching made upon for note cries sleep brings surprise bearlings amazed on anyone shake lumps carelessness mother keep grieved earnestly squealing stopping talons hall ghost then raksha shall miserably whisper soul debt loved left raced dry after cloth presumptuous brings ten umph cold maimed light time a jackal government envied for mistake ground egg gasped thief death warned barrel hate fed scores hark answer dashed sleepy cannot forward are speaks thick shapeless sprung treetop white runs accepted tops plan dreaming'

In [ ]:
generate_text("the child mowgli are playing on the tree", 100, 10)

1/1 [==============================] - 0s 23ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 22ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 21ms/step

1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 21ms/step

1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 19ms/step

1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 20ms/step

1/1 [==============================] - 0s 20ms

'the child mowgli are playing on the tree faithfully art he beasts he while least runs but his here ikki tabaqui mowgli there should there illo said bagheera my see would listen nothing only from as us kill up down near hung i i art through i my there shere remember see himself here what play mowgli speak see honey nor listen why i next out about what if not went shere their so about come what could me were we wise that their being except of go didst might that so shameless as he we strong half that thy shere any lazy it sought still once his'

In [ ]:
for idx in predict_next_word("mowgli on the tree", 50):
    print(unique_tokens[idx],end=' ')

1/1 [==============================] - 0s 21ms/step

tonight perfectly one better always things bare tongue foolish even oh lifted brother safe hungry down going strike killed give near something say pushed oho so almost like meant take here sent come folly indeed taught because little tell ten than thee o heard sit off betrayed look any out 